# Stacked Auto Encoder

### Setup notebook

In [ ]:
import sys, os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torchvision

In [ ]:
# check if module is imported
def assert_imported(package):
    mods = [m.__name__ for m in sys.modules.values() if m]
    assert(package in mods)

In [ ]:
# Importing the dataset from any path in cwd
from pathlib import Path

def read_file(filename, **kwargs):
    assert_imported('pathlib')
    fpath = list(Path('.').glob('**/' + str(filename)))[0]
    return pd.read_csv(fpath, **kwargs)


In [ ]:
# Importing the dataset
movies = read_file('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = read_file('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = read_file('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [ ]:
# Preparing the training set and the test set
training_set = read_file('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = read_file('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [ ]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [ ]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [ ]:
# Creating the architecture of the Stacked Auto Encoder
# inherit from Class nn
class SAE(nn.Module):
    def __init__(self, n_hl1, n_hl2, n_hl3):
        # initialize nn.Module(super of SAE)
        super(SAE, self).__init__()
        
        self.fc1 = nn.Linear(nb_movies, n_hl1)
        self.fc2 = nn.Linear(n_hl1, n_hl2)
        self.fc3 = nn.Linear(n_hl2, n_hl3)
        self.fc4 = nn.Linear(n_hl3, nb_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x


In [ ]:
# create NN
sae = SAE(20, 10, 20)

# define loss function
loss_fn = nn.MSELoss()

# define optimizer
def optimizer(name):
    opt = {'RMS': 'optim.RMSprop(sae.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0.5)', 
         'Adam': 'optim.Adam(sae.parameters(), lr=0.01, eps=1e-08, weight_decay=0.25)'}
    return eval(opt[name])

In [ ]:
# Train the SAE
N_EPOCH = 20

for epoch in range(1, N_EPOCH + 1):
    train_loss = 0
    s = 0.
    
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0) # [torch.FloatTensor of size 1x1682]
        target = input.clone()
        
        # train rated movies only
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False # turn off gradient computation
            output[target == 0] = 0 # save computation cost
            
            # compute (MSE) loss
            # adjust trained/rated movies loss to all movies loss
            loss = loss_fn(output, target)
            mean_adjust = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            train_loss += np.sqrt(loss.data[0] * mean_adjust)
            s += 1.
            
            # define loss direction and optimizer generator
            loss.backward()
            optimize = optimizer("Adam")
            optimize.step()
            
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))